- Sunday, October 21, 2018

# Plotly like a Pro
- The purpose of this notebook is to display some of my most frequently used plotly plotting patterns.

# Lets begin!
***

### First are libraries, these include the usual suspects:
    - pandas
    - numpy
    
### Second are some specitic module imports from plotly to streamline useage
    - init_notebook_mode: necessary to view plotly plots in a jupyter notebook
        - must include `init_notebook_mode(connected=True)` or else plots will not be visible!
    - plotly.graph_objs: the modules are imported directly, so no need to call the typical `py.` prefix

In [1]:
import pandas as pd
import numpy as np


from plotly.offline import init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

## Before we can create any plots, we need to get some data first!
    - I have game & view data scraped from Twitch.tv every ~30 minutes for the month of june 2018
    - Second is a dataset of NYC bus breakdowns and delays (courtesy of Kaggle)

In [2]:
twitch = pd.read_csv('Data/twitch_scrape.csv',encoding='utf-8') # Import csv file as DataFrame

twitch.head() # Display first 10 records

,batch,game,views
0,2018-06-03 00:23:00,Fortnite,235619
1,2018-06-03 00:23:00,Dota 2,102896
2,2018-06-03 00:23:00,League of Legends,91233
3,2018-06-03 00:23:00,IRL,51908
4,2018-06-03 00:23:00,Hearthstone,46972


## Its useful to set the timestamp ('batch') field as the DataFrame's index
- This simplifies aggregating over different time periods - hourly/daily/weekly/etc

In [3]:
twitch = twitch.set_index('batch') #change dataframe index to the timestamp field
twitch.index = pd.to_datetime(twitch.index) # set index dtype to datetime64
twitch.head()

,game,views
batch,,
2018-06-03 00:23:00,Fortnite,235619
2018-06-03 00:23:00,Dota 2,102896
2018-06-03 00:23:00,League of Legends,91233
2018-06-03 00:23:00,IRL,51908
2018-06-03 00:23:00,Hearthstone,46972


## Some basic info
- How many batches are there?
- How many days does this include?
- How batches per day?
    - on avg?
    - max?
    - min?


In [43]:
# How many batches are there?
print "There are {0} separate batches included in the dataframe 'twitch'\n\n".format(twitch.index.nunique())

# How many days does this include?
print "There are {0} separate days included in the dataframe 'twitch'\n".format(twitch.resample('D').count().index.shape[0])

# How batches per day?
print "\nThe most batches scraped in a single day is - {0}".format(int(twitch.to_period('D').index.value_counts().apply({'Max':np.max,'Min':np.min,'Avg':np.mean})['Max']))
print "\nThe least batches scraped in a single day is - {0}".format(int(twitch.to_period('D').index.value_counts().apply({'Max':np.max,'Min':np.min,'Avg':np.mean})['Min']))
print "\nThere are {0} batches per day on average in the dataframe 'twitch'".format(int(twitch.to_period('D').index.value_counts().apply({'Max':np.max,'Min':np.min,'Avg':np.mean})['Avg']))


There are 1785 separate batches included in the dataframe 'twitch'


There are 23 separate days included in the dataframe 'twitch'


The most batches scraped in a single day is - 119991

The least batches scraped in a single day is - 9686

There are 95422 batches per day on average in the dataframe 'twitch'


## Plot 1: Number of batches scraped per day

In [47]:
batches_per_day = twitch.resample('D').count()['game'] # results are a Pandas Series
type(batches_per_day)

pandas.core.series.Series

### This Plotly 'structure' closely matches the documentation
- Note that `Bar`, `Layout`, `Figure` , and `iplot` are called directly because they have been imported without a namespace prefix like `py`
    - Also, most of these objects are plain old dictionaries, so they dont need to be called at all!

In [259]:
data = [Bar(x=batches_per_day.index,
            y=batches_per_day.values,
            opacity=.7)]

layout = Layout(title="Twitch.tv Scraped batches per day")

figure = Figure(data=data,layout=layout)

#iplot(figure)
#############################################################

## more dictionaries!

data = [{'type':'bar',
         'x':batches_per_day.index,
         'y':batches_per_day.values,
         'opacity':.7}]

layout = {'title':"Twitch.tv Scraped batches per day"}

figure = {'data':data,'layout':layout}

#iplot(figure)
############################################################

## Combine objects
#- Only need to call `iplot`

iplot(
        {
            'data':[
                    {'type':'bar',
                     'x':batches_per_day.index,
                     'y':batches_per_day.values,
                     'opacity':.7}
            ],
            'layout':{'title':"Twitch.tv Scraped batches per day"}
        }
)

## Lets change the plot from bar to scatter
    - type needed to be changed from `bar` to `scatter`
    - `mode` key:value pair allows users to select if they want markers (points), lines, or both

In [71]:
iplot(
        {
            'data':[
                    {'type':'scatter',
                     'x':batches_per_day.index,
                     'y':batches_per_day.values,
                     'opacity':.7,
                     'mode':'markers+lines'}
            ],
            'layout':{'title':"Twitch.tv Scraped batches per day"}
        }
)

## Bars and Scatter
- combining different type plots is as simple as combining their respective data fields
- Note: Including the `names` key:value pair lets you customize the name of each plot for the axis labels and the legend
    - If the `names` field is omitted, the legend will default to trace0, trace1, ... tracen

In [72]:
iplot(
        {
            'data':[
                    {'type':'scatter',
                     'x':batches_per_day.index,
                     'y':batches_per_day.values,
                     'opacity':.7,
                     'mode':'markers+lines'},
                {'type':'bar',
                     'x':batches_per_day.index,
                     'y':batches_per_day.values,
                     'opacity':.7}
            ],
            'layout':{'title':"Bars and Scatterplot<br>Twitch.tv Scraped batches per day"}
        }
)

## So far so good. But we can do better.
- Customize colors - htmlcolorcodes.com is a good place to start
- Add trace names
- adjust size of scatter line
- add axis labels
- customize text fields with HTML tags

In [101]:
colorcodes = "#48C9B0 #2037C8 #20C86C #4DA8D9 #0A0D6F #DAF7A6 #FFC300 #FF5733 #C70039".split()

In [118]:
iplot(
        {
            'data':[
                    {'type':'scatter',
                     'x':batches_per_day.index,
                     'y':batches_per_day.values,
                     'opacity':.7,
                     'mode':'markers+lines',
                     'name':'Scrapes per day - Scatter',
                     'marker':{'line':{'width':5,
                                       'color':colorcodes[0]},
                              'size':9,
                              'color':colorcodes[1]}},
                    {'type':'bar',
                     'x':batches_per_day.index,
                     'y':batches_per_day.values,
                     'opacity':.4,
                     'name':'Scrapes per day - Bar',
                     'marker':{'color':colorcodes[3]}}
            ],
            'layout':{'title':"<i>Bars</i> and <i>Scatterplot</i><br><b>Twitch.tv</b><br>Scraped batches per day",
                     'xaxis':{'title':'<b>Days'},'yaxis':{'title':'<b>Frequency'}}
        }
)

## Now let's look at the top 3 games

In [119]:
top_3_games = twitch                                \
                .groupby('game')                      \
                .mean()                               \
                .sort_values('views',ascending=False) \
                .index[:3].tolist()
top_3_games

[u'Fortnite', u'League of Legends', u'Dota 2']

### How do their views change over time?
- Unlike grammar of graphics style plotting libraries, you need to manually specify any particular values you wish to subset across
- For example, we are only interested in plotting the timeseries values for: Fortnite, League of Legends, and Dota2
    - In plotly, we must specify a data object for each of these games. We can manually assign these values to multiple separate DataFrames
    - When the number of traces increases sufficiently, it becomes inpractical to create a separate DataFrame for each value, but this is a prime use case for list comprehension

In [156]:
gamecols = dict(zip(top_3_games,np.random.choice(colorcodes,3,False))) # randomly assign a color from the list of colors

iplot(
        {
            'data':[
                    {'type':'scatter',
                     'x':twitch[twitch['game'] == game].sort_index().index,
                     'y':twitch[twitch['game'] == game].sort_index()['views'],
                     'opacity':.7,
                     'mode':'markers+lines',
                     'name':game,
                     'marker':{'line':{'width':5,
                                       'color':gamecols[game]},
                              'size':1,
                              'color':gamecols[game]}} for game in top_3_games
                   
            ],
            'layout':{'title':"Top 3 twitch games<br>Views over time",
                     'xaxis':{'title':'<b>Days'},'yaxis':{'title':'<b>Frequency'}}
        }
)

In [263]:
## How about Daily?
gamecols = dict(zip(top_3_games,np.random.choice(colorcodes,3,False))) # randomly assign a color from the list of colors


_ = [iplot(
        {
            'data':[
                    {'type':'scatter',
                     'x':twitch[twitch['game'] == game]\
                                 .resample('D')['views']\
                                 .agg([np.max,np.sum,np.mean,np.min])\
                                 .rename(columns={'amax':'Max','amin':'Min','mean':'Average','sum':'Total'})\
                                 .sort_index().index,
                     'y':twitch[twitch['game'] == game]\
                                 .resample('D')['views']\
                                 .agg([np.max,np.sum,np.mean,np.min])\
                                 .rename(columns={'amax':'Max','amin':'Min','mean':'Average','sum':'Total'})\
                                 .sort_index()[AGG],
                     'opacity':.7,
                     'mode':'markers+lines',
                     'name':"{0} - {1}".format(game,AGG),
                     'marker':{'line':{'width':5,
                                       'color':gamecols[game]},
                              'size':2,
                              'color':gamecols[game]}} 
                for game in top_3_games      
            ],
            'layout':{'title':"Top 3 twitch games<br>Views over time - <b>Daily</b>",
                     'xaxis':{'title':'<b>Day'},'yaxis':{'title':'<b>Views'}}
        })
     for AGG in ['Max','Min','Average','Total']]

In [260]:
## How about weekly?
gamecols = dict(zip(top_3_games,np.random.choice(colorcodes,3,False))) # randomly assign a color from the list of colors


_ = [iplot(
        {
            'data':[
                    {'type':'scatter',
                     'x':twitch[twitch['game'] == game]\
                                 .resample('W')['views']\
                                 .agg([np.max,np.sum,np.mean,np.min])\
                                 .rename(columns={'amax':'Max','amin':'Min','mean':'Average','sum':'Total'})\
                                 .sort_index().index,
                     'y':twitch[twitch['game'] == game]\
                                 .resample('W')['views']\
                                 .agg([np.max,np.sum,np.mean,np.min])\
                                 .rename(columns={'amax':'Max','amin':'Min','mean':'Average','sum':'Total'})\
                                 .sort_index()[AGG],
                     'opacity':.7,
                     'mode':'markers+lines',
                     'name':"{0} - {1}".format(game,AGG),
                     'marker':{'line':{'width':5,
                                       'color':gamecols[game]},
                              'size':1,
                              'color':gamecols[game]}} 
                for game in top_3_games      
            ],
            'layout':{'title':"Top 3 twitch games<br>Views over time - <b>Weekly</b>",
                     'xaxis':{'title':'<b>Weeks'},'yaxis':{'title':'<b>Views'}}
        })
     for AGG in ['Max','Min','Average','Total']]

In [243]:
## How about hourly?
gamecols = dict(zip(top_3_games,np.random.choice(colorcodes,3,False))) # randomly assign a color from the list of colors



_ = [iplot(
        {
            'data':[
                    {'type':'scatter',
                     'x':twitch[twitch['game'] == game]\
                                 .groupby(twitch[twitch['game'] == game].index.hour)\
                                 .agg([np.max,np.sum,np.mean,np.min])\
                                 .rename(columns={'amax':'Max','amin':'Min','mean':'Average','sum':'Total'})\
                                 .sort_index().index,
                     'y':twitch[twitch['game'] == game]\
                                 .groupby(twitch[twitch['game'] == game].index.hour)\
                                 .agg([np.max,np.sum,np.mean,np.min])\
                                 .rename(columns={'amax':'Max','amin':'Min','mean':'Average','sum':'Total'})\
                                 .sort_index()['views'][AGG],
                     'opacity':.7,
                     'mode':'markers+lines',
                     'name':"{0} - {1}".format(game,AGG),
                     'marker':{'line':{'width':5,
                                       'color':gamecols[game]},
                              'size':1,
                              'color':gamecols[game]}} 
                for game in top_3_games      
            ],
            'layout':{'title':"Top 3 twitch games<br>Views over time - <b>Hourly</b>",
                     'xaxis':{'title':'<b>Hours'},'yaxis':{'title':'<b>Views'}}
        }
) for AGG in ['Max','Min','Average','Total']]

## Another personal favorite - the humble Boxplot

In [242]:
# Top 10 games
top_10_games = twitch                                \
                .groupby('game')                      \
                .mean()                               \
                .sort_values('views',ascending=False) \
                .index[:10].tolist()
top_10_games
colorcodes= "#F5B7B1 #D7BDE2 #A9CCE3 #AED6F1 #A3E4D7 #A9DFBF #FAD7A0 #F5CBA7 #E5E7E9 #AEB6BF".split()
gamecols = dict(zip(top_10_games,np.random.choice(colorcodes,10,False))) # randomly assign a color from the list of colors


iplot(
        {
            'data':[
                    {'type':'box',
                     'x':twitch[twitch['game'] == game].game,
                     'y':twitch[twitch['game'] == game]['views'],
                     'opacity':.7,
                     'name':"{0}".format(game),
                     'boxpoints':'all',
                     'marker':{'color':gamecols[game]}} 
                for game in top_10_games      
            ],
            'layout':{'title':"Top 10 twitch games</b>",
                     'xaxis':{'title':'<br><br><b>Game'},'yaxis':{'title':'<b>Views'},
                     'margin':{'b':150}}
        }
)

## Change bar order by median
- In this example, the easiest way to re-order the bar plots is to re-order the `top_10_games` list by median
- The second approach is to set the `game` field as type Categorial where the values are ordered by median
- Use `margin` key:value parameter to add boarder space for labels that get cutoff
    - in this case, increase the bottom margin 150 pixels: `'layout':{'margin':{'b':150})`

In [293]:
# Re-order top 10 games (by avg views) by median views in ascending order
twitch[twitch['game'].isin(top_10_games)].groupby('game').median().sort_values('views')

,views
game,
E3 2018,598
Bangai-O!,6517
Realm Royale,25817
Hearthstone,37076
PLAYERUNKNOWN'S BATTLEGROUNDS,41489
IRL,53025
Dota 2,61332
Wacky Races,67714
League of Legends,96513


In [256]:
top_10_games_median = twitch[twitch['game'].isin(top_10_games)].groupby('game').median().sort_values('views').index
gamecols = dict(zip(top_10_games_median,np.random.choice(colorcodes,10,False))) # randomly assign a color from the list of colors



iplot(
        {
            'data':[
                    {'type':'box',
                     'x':twitch[twitch['game'] == game].game,
                     'y':twitch[twitch['game'] == game]['views'],
                     'opacity':.7,
                     'name':"{0}".format(game),
                     'boxpoints':'all',
                     'marker':{'color':gamecols[game]}} 
                for game in top_10_games_median      
            ],
            'layout':{'title':"<b>Top 10 twitch games - by Average</b><br>Ordered by Median",
                     'xaxis':{'title':'<br><br><b>Game'},'yaxis':{'title':'<b>Views'},
                     'margin':{'b':150}}
        }
)

## Max views by day
- fix overlapping axis labels with `dtick` key:value paramter
    - set `'xaxis':{'dtick':'2'}` to display every other x axis tick (when set as a datetime)

In [291]:
iplot(
        {
            'data':[
                    {'type':'bar',
                     'x':twitch.resample('D').max().index.map(lambda day: "June {}<br>{}".format(day.day,day.day_name()[:3])),
                     'y':twitch.resample('D')['views'].max(),
                     'opacity':.7,} 
            ],
            'layout':{'title':"Daily Max Views</b>",
                     'xaxis':{'title':'<b>Day','dtick':'2'},'yaxis':{'title':'<b>Views'},
                     }        }
)

## Bus Breakdown Questions:
- Are bus breakdowns increasing or decreasing year to year? in duration and quantity?
- Are bus breakdowns seasonal? When?
- Which borough has the most bus breakdowns? relative and absolute?
- which bus number has the most breakdowns? relative and absolute?
- Are busses that service multiple schools more susceptible to breakdowns?
- Which route has the most breakdowns? is it seasonal?
- Which bus company has the most breakdowns? which has the least?
- Which bus company is the worst at notifying parents of delays?
- What is the longest delay? when did it occur?
- what is the most frequent reason for delays?

In [294]:
busbreak = pd.read_csv('Data/bus-breakdown-and-delays.csv',encoding='utf-8')

/Users/scott/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.



In [324]:
for _,col in enumerate(busbreak.columns):
    print _,col,busbreak[col].dtype

0 School_Year object
1 Busbreakdown_ID int64
2 Run_Type object
3 Bus_No object
4 Route_Number object
5 Reason object
6 Schools_Serviced object
7 Occurred_On object
8 Created_On object
9 Boro object
10 Bus_Company_Name object
11 How_Long_Delayed object
12 Number_Of_Students_On_The_Bus int64
13 Has_Contractor_Notified_Schools object
14 Has_Contractor_Notified_Parents object
15 Have_You_Alerted_OPT object
16 Informed_On object
17 Incident_Number object
18 Last_Updated_On object
19 Breakdown_or_Running_Late object
20 School_Age_or_PreK object


In [322]:
# set NAN incident number to -1
busbreak.loc[busbreak['Incident_Number'].isna(),'Incident_Number'] = -1
# set incident number type to object
busbreak['Incident_Number'] = busbreak['Incident_Number'].astype(str) 

In [337]:
# drop Created On field - offers no additional insights
busbreak = busbreak.drop('Created_On',axis=1)
busbreak.head()

,School_Year,Busbreakdown_ID,Run_Type,Bus_No,Route_Number,Reason,Schools_Serviced,Occurred_On,Boro,Bus_Company_Name,How_Long_Delayed,Number_Of_Students_On_The_Bus,Has_Contractor_Notified_Schools,Has_Contractor_Notified_Parents,Have_You_Alerted_OPT,Informed_On,Incident_Number,Last_Updated_On,Breakdown_or_Running_Late,School_Age_or_PreK
0,2015-2016,1212699,Special Ed AM Run,48186,N758,Other,75485,2015-09-02T06:27:00,Nassau County,"BORO TRANSIT, INC.",25 minutes,0,Yes,No,No,2015-09-02T06:29:00,-1,2015-09-02T06:29:16,Running Late,School-Age
1,2015-2016,1212700,Special Ed AM Run,2518,L530,Mechanical Problem,21854,2015-09-02T06:24:00,Brooklyn,"RELIANT TRANS, INC. (B232",NaN,0,Yes,Yes,Yes,2015-09-02T06:30:00,-1,2015-09-02T06:30:19,Breakdown,School-Age
2,2015-2016,1212701,Special Ed AM Run,235,K168,Other,18366,2015-09-02T06:45:00,Brooklyn,"NEW DAWN TRANSIT, LLC (B2",30MINS,0,Yes,Yes,No,2015-09-02T06:47:00,-1,2015-09-02T08:05:39,Running Late,School-Age
3,2015-2016,1212703,Special Ed AM Run,2102,K216,Other,21501,2015-09-02T06:55:00,Brooklyn,EMPIRE STATE BUS CORP.,20 min,1,Yes,Yes,No,2015-09-02T07:02:00,-1,2015-09-02T07:02:01,Running Late,School-Age
4,2015-2016,1212704,Special Ed AM Run,48162,N861,Mechanical Problem,75485,2015-09-02T06:55:00,Nassau County,"BORO TRANSIT, INC.",30 min,0,Yes,Yes,No,2015-09-02T07:04:00,-1,2015-09-02T07:04:25,Running Late,School-Age


In [338]:
# update timestamp fields as Pandas Datetime
for timefield in ["Occurred_On","Informed_On","Last_Updated_On"]:
    busbreak[timefield] = pd.to_datetime(busbreak[timefield])


In [349]:
busbreak['Informed_gap'] = (busbreak['Informed_On'] - busbreak['Occurred_On'])

## cleanup field `How long delayed`

In [424]:
def bucket_delay(delay):
    def dc(delay):
        # strip out 'Minutes' and any variation by removing all non-digit or '-' characters
        strip_letters = ''.join([x for x in list(delay) if (x.isdigit() or x == "-")])
        return strip_letters if len(strip_letters) > 0 else "-1"

    delay = dc(delay)
    delay_strip = str(delay) if len(delay.split('-')) == 1 else (delay.split('-')[0])
    if delay_strip == '':
        return 'No Value'
    elif int(delay_strip) < 16:
        return "0-15"
    elif int(delay_strip) < 31:
        return "16-30"
    elif int(delay_strip) < 46:
        return "31-45"
    elif int(delay_strip) < 61:
        return "46-60"
    elif int(delay_strip) < 120:
        return "61-120"
    else:
        return "other"

In [426]:
busbreak['How_Long_Delayed'] = busbreak['How_Long_Delayed'].astype(str).map(bucket_delay)

## Are bus breakdowns increasing or decreasing year to year? in duration and quantity?

In [495]:
busbreak = busbreak.loc[busbreak['School_Year'] != '2019-2020',:] # drop value from 2019-2020

# limit to full school years
full_schoolyear = ['2015-2016','2016-2017','2017-2018']

buscolors = "#A569BD #D98880 #F5B7B1 #D7BDE2 #A9CCE3 #AED6F1 #A3E4D7 #A9DFBF #FAD7A0 #F5CBA7 #E5E7E9 #AEB6BF".split()

borocols = dict(zip(busbreak.Boro.unique(),np.random.choice(buscolors,12,False))) # randomly assign a color from the list of colors

data = []

data.append({'type':'scatter',
                         'x':busbreak[(busbreak.School_Year.isin(full_schoolyear))].groupby('School_Year').count().index,
                         'y':busbreak[(busbreak.School_Year.isin(full_schoolyear))].groupby('School_Year').count()['Bus_No'],
                 'opacity':.9,
                 'name':'',
                 'mode':'markers+lines',
                 'marker':{'line':{'width':5,
                                   'color':borocols.values()[5]},
                           'size':2,
                           'color':borocols.values()[5]},
                  'showlegend':False})
data.append({'type':'bar',
                         'x':busbreak[(busbreak.School_Year.isin(full_schoolyear))].groupby('School_Year').count().index,
                         'y':busbreak[(busbreak.School_Year.isin(full_schoolyear))].groupby('School_Year').count()['Bus_No'],
                 'opacity':.7,
                 'name':'',
                 'marker':{'color':borocols.values()[4]},
                 'showlegend':False})

iplot(
        {
            'data':data,
            'layout':{'title':"Schoolbus Delays by School Year</b>",
                     'xaxis':{'title':'<b>School Year',},'yaxis':{'title':'<b>Delays'},
                     }        }
)

## There is clearly year over year increase for schoolbus delays across
- Note: the pace of increase slowed down from 2016-2017 - 2017-2018

### How does does this look on a Boro by Boro basis?

In [496]:
data = []
for b in busbreak.Boro.unique():
    data.append({'type':'scatter',
                             'x':busbreak[(busbreak.School_Year.isin(full_schoolyear)) & (busbreak.Boro == b)].groupby('School_Year').count().index,
                             'y':busbreak[(busbreak.School_Year.isin(full_schoolyear)) & (busbreak.Boro == b)].groupby('School_Year').count()['Bus_No'],
                     'opacity':.7,
                     'name':b,
                     'mode':'lines',
                     'marker':{'line':{'width':5,
                                       'color':borocols[b]},
                               'size':1,
                               'color':borocols[b]}})
    data.append({'type':'bar',
                             'x':busbreak[(busbreak.School_Year.isin(full_schoolyear)) & (busbreak.Boro == b)].groupby('School_Year').count().index,
                             'y':busbreak[(busbreak.School_Year.isin(full_schoolyear)) & (busbreak.Boro == b)].groupby('School_Year').count()['Bus_No'],
                     'opacity':.7,
                     'name':b,
                     'marker':{'color':borocols[b]},
                     'showlegend':False})


iplot(
        {
            'data':data,
            'layout':{'title':"Schoolbus Delays by School Year</b>",
                     'xaxis':{'title':'<b>School Year',},'yaxis':{'title':'<b>Delays'},
                     }        
        }
)

## The worst offender of Bus delays is Manhattan which shot past the Bronx for the 2017-2018 School Year

In [539]:
schoolyear_delays = busbreak.pivot_table(values='Bus_No',columns='How_Long_Delayed',index=['School_Year'],aggfunc=np.size)
schoolyear_delays.columns.name = 'Delay in Min'
schoolyear_delays

Delay in Min,0-15,16-30,31-45,46-60,61-120,No Value,other
School_Year,,,,,,,
2015-2016,17450.0,27937.0,5633.0,1417.0,275.0,9830.0,642.0
2016-2017,19796.0,39526.0,10260.0,835.0,123.0,12015.0,586.0
2017-2018,11291.0,30997.0,22889.0,10728.0,3187.0,10326.0,15.0
2018-2019,1596.0,3998.0,6246.0,2817.0,1158.0,1644.0,NaN


In [545]:
iplot(
        {
            'data':[{'type':'scatter',
                    'x':schoolyear_delays[schoolyear_delays.index.isin(full_schoolyear)][col].index,
                    'y':schoolyear_delays[schoolyear_delays.index.isin(full_schoolyear)][col].values,
                    'opacity':.7,
                    'name':"{} Minutes".format(col) if col not in ['No Value','other'] else col}
                   for col in schoolyear_delays.columns],
            'layout':{'title':"Schoolbus Delays by School Year</b>",
                     'xaxis':{'title':'<b>School Year',},'yaxis':{'title':'<b>Delays'},
                     }        
        }
)

## Sadly the Shortest delays in duration are decreasing, while the larger duration delays all have a substantial increase going into 2017-2018

### How does this picture change across Boros?
    - Bronx
    - Manhattan
    - Brooklyn
    - Queens
    - Staten Island
    - Westchester (County)
    - Rockland County

In [559]:
boro7 = ['Bronx','Manhattan','Brooklyn','Queens','Staten Island','Westchester','Rockland County']

In [579]:
boro_delays = busbreak.pivot_table(values='Bus_No',columns='How_Long_Delayed',index=['School_Year','Boro'],aggfunc=np.size)
boro_delays.loc[(full_schoolyear,boro7),:]

How_Long_Delayed               0-15    16-30    31-45   46-60  61-120  \
School_Year Boro                                                        
2015-2016   Bronx            7066.0   7104.0    655.0   168.0    13.0   
            Brooklyn         2609.0   6797.0   2634.0   600.0   122.0   
            Manhattan        2594.0   6150.0    825.0   300.0    88.0   
            Queens           1432.0   4452.0    967.0   240.0    36.0   
            Rockland County    81.0    124.0     13.0     2.0     1.0   
            Staten Island    1619.0    771.0     93.0    21.0     7.0   
            Westchester       926.0    837.0     94.0     9.0     2.0   
2016-2017   Bronx            7649.0  10598.0   1131.0   136.0    43.0   
            Brooklyn         2807.0   8926.0   4476.0   274.0    28.0   
            Manhattan        3042.0  10096.0   1805.0   246.0    33.0   
            Queens           1767.0   4486.0   1695.0    96.0     8.0   
            Rockland County   104.0     94.0     22.0     2.0     2.0   
            Staten Island    1673.0   1997.0    179.0    19.0     1.0   
            Westchester      1202.0    821.0    201.0    18.0     2.0   
2017-2018   Bronx            5350.0   6582.0   2957.0  1290.0  1065.0   
            Brooklyn         1823.0   6990.0   3648.0  2429.0  1006.0   
            Manhattan        1182.0   6101.0  12352.0  2236.0   458.0   
            Queens           1069.0   3609.0   2350.0  1980.0   301.0   
            Rockland County    17.0    105.0    111.0    54.0     4.0   
            Staten Island     498.0   4570.0    304.0   477.0    24.0   
            Westchester       120.0   1074.0    303.0   185.0   202.0   

How_Long_Delayed             No Value  other  
School_Year Boro                              
2015-2016   Bronx              2094.0   20.0  
            Brooklyn           2713.0  164.0  
            Manhattan          1423.0  190.0  
            Queens             2703.0   44.0  
            Rockland County       9.0    5.0  
            Staten Island       251.0    3.0  
            Westchester          44.0  201.0  
2016-2017   Bronx              3767.0   65.0  
            Brooklyn           2902.0  224.0  
            Manhattan          1348.0  191.0  
            Queens             2950.0   48.0  
            Rockland County      21.0    1.0  
            Staten Island       239.0    6.0  
            Westchester          87.0   12.0  
2017-2018   Bronx              3036.0    2.0  
            Brooklyn           2468.0    3.0  
            Manhattan          1058.0    3.0  
            Queens             2980.0    5.0  
            Rockland County      14.0    NaN  
            Staten Island       121.0    NaN  
            Westchester          46.0    1.0

In [511]:
busbreak[(busbreak.School_Year.isin(full_schoolyear))]\
        .groupby(['School_Year','How_Long_Delayed'])\
        .count()['Bus_No'].reset_index()\
        .rename(columns={'How_Long_Delayed':'Delay','Bus_No':'Frequency'})\
        .sort_values(['School_Year','Frequency'],ascending=[1,0])

,School_Year,Delay,Frequency
1,2015-2016,16-30,27937
0,2015-2016,0-15,17450
5,2015-2016,No Value,9830
2,2015-2016,31-45,5633
3,2015-2016,46-60,1417
6,2015-2016,other,642
4,2015-2016,61-120,275
8,2016-2017,16-30,39526
7,2016-2017,0-15,19796
12,2016-2017,No Value,12015


In [499]:
busbreak.How_Long_Delayed

0            16-30
1         No Value
2            16-30
3            16-30
4            16-30
5         No Value
6         No Value
7           61-120
8            16-30
9            16-30
10           16-30
11            0-15
12           16-30
13            0-15
14        No Value
15           16-30
16           16-30
17        No Value
18           16-30
19            0-15
20           16-30
21        No Value
22            0-15
23           16-30
24           16-30
25            0-15
26            0-15
27           16-30
28            0-15
29           16-30
            ...   
253188    No Value
253189       46-60
253190       16-30
253191       46-60
253192       46-60
253193       16-30
253194       46-60
253195       31-45
253196       31-45
253197       16-30
253198       46-60
253199       46-60
253200    No Value
253201    No Value
253202    No Value
253203       31-45
253204       16-30
253205    No Value
253206       31-45
253207       31-45
253208       46-60
253209      

In [334]:
busbreak.Schools_Serviced.value_counts().reset_index().rename(columns={'index':'School_id','Schools_Serviced':'Frequency'})['School_id'].map(lambda x: len(x.split(','))).value_counts()

2     4601
3     3316
1     2990
4     1597
5      639
6      358
7      218
8      155
9       99
10      41
11      27
12      18
13       9
14       2
Name: School_id, dtype: int64